# Simulation Example

Here we demonstrate how to use this package to estimate the multivariate PSD of VAR(2) simulated data.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sgvb_psd.utils.sim_varma import SimVARMA

np.random.seed(0)

var2 = SimVARMA(
    n_samples=1024,
    var_coeffs=np.array([[[0.5, 0.], [0., -0.3]], [[0., 0.], [0., -0.5]]]),
    vma_coeffs=np.array([[[1., 0.], [0., 1.]]]),
    sigma=np.array([[1., 0.9], [0.9, 1.]])
)
var2

2024-09-19 16:44:47.530063: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-19 16:44:47.540264: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 16:44:47.614449: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 16:44:47.688938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 16:44:47.751000: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

16:44:49|SGVB-PSD|INFO| No GPU detected. Running on CPU.


0.50,0.00
0.00,-0.30
0.00,0.00
0.00,-0.50
1.00,0.00
0.00,1.00
1.00,0.90
0.90,1.00


In [2]:
var2.plot(
    off_symlog=False,
    xlims=[0, np.pi]
)
plt.savefig(f'var2_data.png')
plt.close();

![VAR(2) Data](var2_data.png)

In [3]:
from sgvb_psd.psd_estimator import PSDEstimator

optim = PSDEstimator(
    x=var2.data,
    N_theta=30, nchunks=1, 
    ntrain_map=1000,  
    max_hyperparm_eval=5,
    fs=2*np.pi,
)
optim.run(lr=0.003)
_ = optim.plot(
    true_psd=[var2.psd, var2.freq], 
    off_symlog=False,
    xlims=[0, np.pi],
);

plt.savefig(f'var2_psd.png');
plt.close()

_ = optim.plot_coherence(
    true_psd=[var2.psd, var2.freq],
    labels='XY'
)
plt.savefig(f'var2_coh.png');
plt.close()

_ = optim.plot_vi_losses()
plt.savefig(f'var2_vi_losses.png');
plt.close()

16:44:52|SGVB-PSD|INFO| Final PSD will be of shape: 512 x 2 x 2
16:44:52|SGVB-PSD|DEBUG| Inputted data shape: (1024, 2)
16:44:52|SGVB-PSD|DEBUG| Model instantiated: SpecPrep(x(t)=(1024, 2), y(f)=(1, 512, 2), Xmat_delta=(512, 32), Xmat_theta=(512, 32))
16:44:52|SGVB-PSD|INFO| Using provided learning rate: 0.003
16:44:52|SGVB-PSD|INFO| Training model
16:44:52|SGVB-PSD|DEBUG| Starting Model Inference Training..
16:44:52|SGVB-PSD|DEBUG| Start Point Estimating... 
16:44:55|SGVB-PSD|DEBUG| MAP Training Time: 3.32s
16:44:55|SGVB-PSD|DEBUG| Start ELBO maximisation... 
16:45:02|SGVB-PSD|DEBUG| VI Time: 6.29s
16:45:02|SGVB-PSD|DEBUG| Total Inference Training Time: 9.85s
16:45:02|SGVB-PSD|INFO| Model trained in 10.06s
16:45:02|SGVB-PSD|INFO| Computing posterior PSDs
16:45:03|SGVB-PSD|INFO| Optimal PSD estimation complete in 0.88s


![VAR(2) PSD](var2_psd.png)

![VAR(2) Coherence](var2_coh.png)

![VAR(2) VI Losses](var2_vi_losses.png)